# <center> ETL PIPELINE WITH APACHE CASSANDRA </center>
<pre><b>
Summary:
    I. ETL Pipeline for Pre-Processing the Files
        1. CSV Files pre-processing
   II. Design Tables for a Specific Set of CQL Queries with Apache Cassandra.
        1. Setting Apache Cassandra Cluster and Keyspace
        2. Creating Tables and Queries
            2.1. Query1
            2.2. Query2
            2.3. Query3
        3. Drop All Tables and Close Out the Sessions
            
        
</b></pre>

## I. ETL Pipeline for Pre-Processing the Files

### 1. CSV Files Pre-Processing

#### Import Python packages 

In [1]:
# Import Python packages
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking the current working directory
print(os.getcwd())

# Getting the current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Creating a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# joining the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
full_data_rows_list = [] 
    
for f in file_path_list:
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# checking the number of rows in the csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


## II. Design Tables for a Specific Set of CQL Queries with Apache Cassandra.

<b> Now we are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>.<br>
 The event_datafile_new.csv contains the following columns: <b>
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

### 1. Setting Apache Cassandra Cluster and Keyspace

#### Creating a Cluster

In [5]:
from cassandra.cluster import Cluster
cluster = Cluster()
session = cluster.connect()

#### Creating a Keyspace

In [6]:
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify
    WITH REPLICATION = {
    'class' : 'SimpleStrategy', 'replication_factor' : 1
    }
""")

#### Setting a Keyspace

In [7]:
session.set_keyspace('sparkify')

### 2. Creating Tables and Queries


Now we need to create tables to run the following queries.<br>
<b> 
1. Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4<br>
2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182<br>
3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'</b>

### 2.1. Query 1: <br>
* Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

<b> Create a query based table</b><br>
<pre>- This table labeled "<b>songs_bysession</b>" will be used to execute <b>Query1</b>. Since this query requires sessionId and itemInSession columns to retrieve the rows from the database, we need to set <b>session_id as a partition key</b> (<b>sessionId</b>  is enough to identify each unique row in the table for this specific query) and <b>itemInSession as a clustering
column</b>.
</pre>

In [8]:
create_table_query1 = """
    CREATE TABLE IF NOT EXISTS songs_bysession(
    sessionId INT,
    itemInSession INT,
    artist VARCHAR,
    song VARCHAR,
    length DOUBLE,
    PRIMARY KEY(sessionId, itemInSession)
    );
"""

session.execute(create_table_query1)

<b> Insert elements into the table<b>

In [9]:
#events file path
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        #insert query
        query = "INSERT INTO songs_bysession (sessionId, itemInSession, artist, song, length) "
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

<b> Query Execution (SELECT statement)<b>

In [10]:
select_query = """
    SELECT artist, song, length FROM songs_bysession
    WHERE sessionId = %s and itemInSession = %s;
    """

result = session.execute(select_query, (338, 4))
for res in result:
    print(res.artist, res.song, res.length)

Faithless Music Matters (Mark Knight Dub) 495.3073


### 2.2. Query 2: <br>
* Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

<b> Create a query based table</b><br>
<pre>- This table labeled "<b>songs_byuser</b>" will be used to execute <b>Query2</b>. Since this query requires userid, sessionid and itemInSession columns to retrieve the rows from the database, we need to set both <b>userId and sessionId as a 
composite partition key</b> (The combination of <b>userId</b> and <b> sessionId</b> are enough to identify each unique row in the table for this specific query) and <b>itemInSession as a clustering column</b>.
</pre>

In [11]:
create_table_query2 = """
    CREATE TABLE IF NOT EXISTS songs_byuser(
    userId INT,
    sessionId INT,
    itemInSession INT,
    artist VARCHAR,
    song VARCHAR,
    firstName VARCHAR,
    lastName VARCHAR,
    PRIMARY KEY((userId, sessionId), itemInSession)
    );
"""

session.execute(create_table_query2)             

<b> Insert elements into the table<b>

In [12]:
#events file path
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        #insert query
        query = "INSERT INTO songs_byuser (userId, sessionId, itemInSession, artist, song, firstName, lastName) "
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

<b> Query Execution (SELECT statement)<b>

In [13]:
select_query = """
    SELECT artist, song, firstName, lastName FROM songs_byuser
    WHERE userId = %s and sessionId = %s;
    """

result = session.execute(select_query, (10, 182))
for res in result:
    print(res.artist, res.song, res.firstname, res.lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### 2.3. Query 3: <br>
* Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

<b> Create a query based table</b><br>
<pre>- This table labeled "<b>users_bysong</b>" will be used to execute <b>Query3</b>. Since this query requires song and userId 
columns to retrieve the rows from the database, we need to set <b>song as a partition key</b> (<b>song</b> is enough to identify each unique row in the table for this specific query) and <b>userId as a clustering column</b>.
</pre>

In [14]:
create_table_query3 = """
    CREATE TABLE IF NOT EXISTS users_bysong(
    song VARCHAR,
    userId INT,
    firstName VARCHAR,
    lastName VARCHAR,
    PRIMARY KEY(song, userId)
    );
"""

session.execute(create_table_query3)

<b> Insert elements into the table<b>

In [15]:
#events file path
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        #insert query
        query = "INSERT INTO users_bysong (song, userId, firstName, lastName) "
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

<b> Query Execution (SELECT statement)<b>

In [16]:
select_query = """
    SELECT firstName, lastName FROM users_bysong
    WHERE song=%s;
    """

result = session.execute(select_query, ('All Hands Against His Own',))
for res in result:
    print (res.firstname, res.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### 3. Drop All Tables and Close Out the Sessions

<b> Drop all tables </b>

In [17]:
session.execute("""DROP TABLE IF EXISTS songs_bysession;""")
session.execute("""DROP TABLE IF EXISTS songs_byuser;""")
session.execute("""DROP TABLE IF EXISTS users_bysong;""")

<b> Close the session and cluster connection <b/>

In [18]:
session.shutdown()
cluster.shutdown()